In [1]:
import random
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import ctypes
from time import sleep
import threading
myappid = 'mycompany.myproduct.subproduct.version'
ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)

0

In [2]:
class poketrainer():
    
    def __init__(self):
        self.inventar = {
            'Gold' : [1000],
            'Trank' : [0,300],
            'Pokball':[0,50],
            'Attacke auffüllen':[0,100]
        }
        self.pok_sack=[]
 
    
    def heile_pok(self,welchespok):
            trainer.pok_sack[int(welchespok)].leb = trainer.pok_sack[int(welchespok)].lebenges
            self.inventar['Trank'][0] -=1
            trainer.pok_sack[int(welchespok)].tot = False
            
    def attacke_auffuellung(self,welchespok):
            for key in  trainer.pok_sack[int(welchespok)].angriffe:
                trainer.pok_sack[int(welchespok)].angriffe[key][0] = trainer.pok_sack[int(welchespok)].angriffe[key][2]
                
            
    def neues_pok(self,ang,vert,leb,name,entw1,entw2,element):
        if element == 'Feuer':
            self.pok_sack.append(Feuer_pok(ang, vert, leb, name, entw1,entw2))
        if element == 'Pflanzen':
            self.pok_sack.append(Pflanzen_pok(ang, vert, leb, name, entw1,entw2))
        if element == 'Wasser':
            self.pok_sack.append(Wasser_pok(ang, vert, leb, name, entw1,entw2))
    
        
    def inventar_anschauen(self):
        invent = ''
        for key in self.inventar:
            invent += key+': '+str(self.inventar[key][0])+'\n'     
        w.label.setText('in deinem Inventar befinden sich \n'+invent)
        w.label.adjustSize()
        
    def hohes_gras(self):
        trainer.sack_anschauen()
        welchespok = input('Welches Pok willst du kämpfen lassen? (Nummer)')
        pokran = trainer.pok_sack[int(welchespok)-1].random_pok_generieren()
        print('Ein wildes ', pokran.nam, ' taucht auf!!!')
        print('Ang.: ', pokran.ang, ' Vert.: ',pokran.vert, ' Leben: ', pokran.leb)
        print()
        trainer.pok_sack[int(welchespok)-1].kampf(pokran)
        
        

trainer = poketrainer()

In [3]:
class shop():
 
    def __init__(self):
          self.items = {
            'Trank' : [0,300],
            'Pokball' : [0,50],
            'Attacke auffüllen' : [0,100] 
        }
        
    def shopping(self,kauf,stück):

        if stück <= self.items[kauf][0] and trainer.inventar['Gold'][0] >= trainer.inventar[kauf][1]*stück:
            trainer.inventar['Gold'][0]-=trainer.inventar[kauf][1]*stück
            trainer.inventar[kauf][0]+=1*stück
            w.label.setText('Du hast '+ str(stück)+ str(kauf)+ ' um '+ str(trainer.inventar[kauf][1]*stück) + ' gekauft')
            w.label.adjustSize()   
        elif stück >= self.items[kauf][0]:
            w.label.setText('So viel habe ich nicht.')
            
        else:
            w.label.setText('Du haben zu wenig Geld...')
        w.label.adjustSize()    
shop1 = shop()    

In [4]:
class pok():
    
    def __init__(self, angriff, verteidigung, leben, name, entw1, entw2):
        self.exp = 0
        self.level = 1
        self.tot = False 
        self.ang = angriff
        self.vert = verteidigung
        self.leb = leben
        self.lebenges = leben
        self.nam = name
        self.entw1 = entw1
        self.entw2 = entw2
        self.expgain = int((angriff + verteidigung + leben*0.3)/3)
        self.krit = random.randint(10,40)
        self.exp_needed = 100
        self.element = 'keines'
        self.angriffe={
            'Hieb':[1000,int(self.ang*0.3),1000],
            'Tritt':[5,int(self.ang*0.7),5],
            'Kopfstoß':[2,int(self.ang),2]      #dict mit möglichen Angriffen, erste Zahl ist
        }                                     # wie oft man den Angriff ausführen kann 
                                              # zweite Zahl in Liste ist die Stärke der Attacke (müsste man wahrscheinlich eher
                                              # von angriff und Vert berechnen als einfach eingeben, damit nicht jeder gleich ist) dritte zahl ist wie weit der Angriff aufgefüllt werden kann
        
    
    def level_up(self):
        self.ang=int(self.ang*1.2)
        self.vert = int(self.vert*1.2)
        self.lebenges = int(self.lebenges*1.2)
        self.exp = 0 
        self.level+=1
        self.leb = self.lebenges
        self.exp_needed = int(self.exp_needed*1.1)
        
    def evolve1(self):
        self.nam = self.entw1
        self.angriff_lernen()
    
    def angriff_lernen(self):
        if self.element == 'Pflanze':
            self.angriffe['Blattwurf'] = [2,int(self.ang*1.2),2]
        elif self.element == 'Feuer':
            self.angriffe['Feuerball'] = [2,int(self.ang*1.2),2]
        elif self.element == 'Wasser':
            self.angriffe['Aquaknarre'] = [2,int(self.ang*1.2),2]
        else:
            self.angriffe['Wutangriff'] = [2,int(self.ang*1.2),2]
        print(self.nam,'hat einen neuen Angriff erlernt!')
            
    def kampf(self, pok2):
        fangbar = random.randint(0,2)
        gefangen = random.randint(0,2)
        if self.tot or pok2.tot:
            print('Kämpfen nicht möglich, eines der Pokemon hat keine Lebenspunkte mehr.')

        while self.tot == False and pok2.tot ==False:
            kritisch1 = random.randint(0,self.krit)
            kritisch2 = random.randint(0,pok2.krit)

            for key in self.angriffe:
                print('Angriff:',key, '  Anzahl:', self.angriffe[key][0],'  Stärke:', self.angriffe[key][1])
            
            ang_auswahl = input('Welchen Angriff willst du verwenden?')    
            
            if ang_auswahl in self.angriffe and kritisch1 != 10: 
                print('Kritischer Angriff!!!')
                pok2.leb = pok2.leb-int(self.angriffe[ang_auswahl][1])
                self.angriffe[ang_auswahl][0] -=1
                schaden = self.angriffe[ang_auswahl][1]
            
            elif ang_auswahl in self.angriffe and kritisch1 == 10: 
                print('Kritischer Angriff!!!')
                pok2.leb = pok2.leb-int(self.angriffe[ang_auswahl][1]*5)
                self.angriffe[ang_auswahl][0] -=1
            
            else:
                print('Dein Pok hat dich nicht verstanden und setzt Hieb ein.')
                pok2.leb = pok2.leb-int(self.angriffe['Hieb'][1])
                ang_auswahl = 'Hieb'

            
            
            if kritisch2 == 10:
                self.leb = int(self.leb - pok2.ang/self.vert*10*5)
            else:
                self.leb = int(self.leb - pok2.ang/self.vert*10)
            
            
            print(pok2.nam, ' hat ', pok2.leb, ' Lebenspunkte (-',self.angriffe[ang_auswahl][1],')')
            print(self.nam, ' hat ', self.leb, ' Lebenspunkte (-',int(pok2.ang/self.vert*10),')')
            print()
            
            
            if self.leb <=0 and pok2.leb<=0:
                if int(self.ang/pok2.vert*10) > int(pok2.ang/self.vert*10):
                    print(pok2.nam, ' hat mit letzter Kraft ', self.nam, ' besiegt.')
                    pok2.leb =10
                else:
                    print(self.nam, ' hat mit letzter Kraft ', pok2.nam, ' besiegt.')
                    self.leb =10
                
            
            if self.leb <= 0:
                pok2.exp = pok2.exp + self.expgain
                self.tot = True
                self.leb = 0
                weiter = input(str(self.nam)+'hat keine Lebenspunkte mehr, willst du weiter kämpfen? (Ja, Nein)')
                if weiter == 'Ja':
                    welchespok = input('Welches Pok willst du kämpfen lassen? (Nummer)')
                    trainer.pok_sack[int(welchespok)-1].kampf(pok2)
                    break
            if pok2.leb <= 0:
                self.exp = self.exp + pok2.expgain
                pok2.tot = True
                pok2.leb = 0
                trainer.inventar['Gold'][0] += int((pok2.ang + pok2.vert + pok2.lebenges*0.5))
                print('Du hast', int((pok2.ang + pok2.vert + pok2.lebenges*0.5)), 'Goldstücke erhalten')
                
        
        if fangbar == 1 and pok2.leb<=0:
            fangen = input(pok2.nam + 'scheint sich zu unterwerfen, willst du veruchen es zu fangen? (Ja, Nein)')
                    
            if fangen == 'Ja' and trainer.pok_ball > 0:
                trainer.pok_ball-=1
                        
                if gefangen == 1:
                    pok2.leb= pok2.lebenges
                    pok2.tot = False
                    trainer.pok_sack.append(pok2)
                    print('Du hast', pok2.nam, 'gefangen!!!')
                            
                else:
                    print('Schade,', pok2.nam, 'hat es sich anders überlegt...')
            elif fangen == 'Nein':
                print(pok2.nam, 'weiß nicht was es tun soll...')
            else:
                print('Keine Pokbälle mehr!')
                print()
        else:
            print(pok2.nam, 'ist geflohen...') 
            print()
                         
        print(self.nam, 'hat jetzt noch ', self.leb, ' Leben')
        print()
          

    def random_pok_generieren(self):
            namen = [['Taubsi','Tauberich','Tauberus'],['Ditto', 'Litto', 'Schmidto'], ['Relaxo', 'Arbeito', 'Hustleo'], ['Raupi', 'Kokooni', 'Schmetterlingi']]
            ang = int(self.ang*random.random()*2)
            vert = int(self.vert*random.random()*2)
            leb = int(self.lebenges*random.random()*1.2) 
            zuf_name = random.randint(0,3)
            pokran = pok(ang, vert, leb, namen[zuf_name][0],namen[zuf_name][1], namen[zuf_name][2])
            return pokran
            

In [5]:
class Feuer_pok(pok):
    def __init__(self,ang, vert, leb, name, entw1,entw2):
        pok.__init__(self,ang, vert, leb, name, entw1,entw2)
        self.element='Feuer'

#Unterklassen nicht notwendig, nur zum probieren und eventuell eigene Funktionen hinzufügen

In [6]:
class Pflanzen_pok(pok):
    def __init__(self,ang, vert, leb, name, entw1,entw2):
        pok.__init__(self,ang, vert, leb, name, entw1,entw2)
        self.element='Pflanze'

In [7]:
class Wasser_pok(pok):
    def __init__(self,ang, vert, leb, name, entw1,entw2):
        pok.__init__(self,ang, vert, leb, name, entw1,entw2)
        self.element='Wasser'

In [8]:
trainer.neues_pok(25,50,120,'Feu','Feufeu','Feufeufeu','Feuer')
trainer.neues_pok(10,15,200,'Bisasam', 'Bisaknosp', 'Bisaflor','Pflanzen')
trainer.neues_pok(10,15,200,'Bisa', 'Bisaknosp', 'Bisaflor','Pflanzen')

In [9]:
class my_thread(threading.Thread):
    def __init__(self, iD, Name):
        threading.Thread.__init__(self)
        self.iD = iD
        self.Name = Name
        
    def run(self):
        self.i=0
        self.running= True
        while self.running:    
            w.label1.move(300,310)
            sleep(0.7)
            w.label1.move(300,300)
            sleep(0.7)
            self.i+=1
        print('ende')
    def stop(self):
        self.running = False


In [10]:
class Fenster(QWidget):
    a=False
    def __init__(self):
        super().__init__()
        self.initMe()
        
        QToolTip.setFont(QFont('Arial', 10))
        self.setGeometry(30,50,1700,900)
        self.setWindowTitle('Bestes Pokemon Game seit es Computer gibt')
        self.setWindowIcon(QIcon('pokball.png'))
    
    def initMe(self):
        
        self.Commands = {
        'Hohes Gras' : trainer.hohes_gras,
        'Inventar' : trainer.inventar_anschauen,
        'Trank einsetzen' : self.trank,
        'Attacke auffuellen' : self.auffuellung,
        'Shop' : self.shopping_gui,}
        #QApplication.setStyle(QStyleFactory.create('Plastique'))
        j = 50
        i = 0
        button = []
        for key in self.Commands:
            button.append(QPushButton(key,self))
            button[i].move(20,j)
            j+=40
            button[i].clicked.connect(self.Commands[key])
            i+=1
        self.label1 = QLabel(self)
        self.label1.setPixmap(QPixmap('pok2_200.png'))
        self.label1.move(300,300)
        button23 = QPushButton('bewege', self)
        button23.clicked.connect(self.start_thread)
        button22 = QPushButton('Ende',self)
        button22.move(400,600)
        button22.clicked.connect(self.stop_thread)
        button[3].setToolTip('schaut den Inventar an')
        button[2].setToolTip('nutzt einen Trank, um die Lebenspunkte deines Pok voll aufzufüllen')
        button[2].move(920,30)
        self.label = QLabel('Was willst du tun?',self)
        self.label.move(200,50)
        self.pok_sack_combo()
        self.anzeigen = QPushButton('Anzeigen', self)
        self.anzeigen.move(800,30)
        self.pok_anzeige = QLabel('Poks',self)
        self.anzeigen.clicked.connect(self.anzeigen_geclicked)
        
        self.show()
    
    def shopping_gui(self):
        self.label.setText('Ich haben du brauchen:')
        self.label.adjustSize()
        self.combo_Box2 = QComboBox(self)
        self.spinbox1 = QSpinBox(self)
        self.kaufen_button = QPushButton('Kaufen', self)
        self.shop_verlassen_button = QPushButton('Shop verlassen', self)
        j = 80
        i=0
        self.artikel_list =[]
        for key in shop1.items:
            shop1.items[key][0] = random.randint(1,5)
        for key in shop1.items:
            self.artikel = QLabel(str(key)+': '+ str(shop1.items[key][0])+' zu je '+ str(shop1.items[key][1])+ ' Goldstücken',self)
            self.artikel_list.append(self.artikel)
            self.combo_Box2.addItem(key)
            self.artikel.move(200,j)
            j += 30
            i+=1
            self.artikel.show()
    

        self.spinbox1.setMinimum(1)
        self.spinbox1.setMaximum(5)
        self.spinbox1.move(360,j+30)
        self.combo_Box2.move(200,j+30)
        self.kaufen_button.move(400,j+30)
        self.shop_verlassen_button.move(510,j+30)
        self.combo_Box2.show()
        self.spinbox1.show()
        self.kaufen_button.show()
        self.shop_verlassen_button.show()
        self.shop_verlassen_button.clicked.connect(self.shop_verlassen)
        self.kaufen_button.clicked.connect(self.kaufen_button_clicked)
        
    def shop_verlassen(self):
        self.label.setText('Was willst du tun?')
        self.label.adjustSize()
        self.kaufen_button.close()
        self.shop_verlassen_button.close()
        self.combo_Box2.close()
        self.spinbox1.close()
        for i in range(0,len(self.artikel_list)):
            self.artikel_list[i].close()
        
    def kaufen_button_clicked(self):
        stück = self.spinbox1.value()
        sache = self.combo_Box2.currentText()
        shop1.shopping(sache, stück)
        
    def auffuellung(self):
        self.index_pok_auffüllen = self.combo_Box.currentIndex()
        if trainer.inventar['Attacke auffüllen'][0]>=1:
            trainer.attacke_auffuellung(self.index_pok_auffüllen)
            self.label.setText('Attacken wurden aufgefüllt.')
            self.label.adjustSize()
        else:
            self.label.setText('Zu wenig Auffüllungen.')
            self.label.adjustSize()
            
    def anzeigen_geclicked(self):
        self.x = self.combo_Box.currentText() 
        for i in range(0,len(trainer.pok_sack)):
            if self.x == trainer.pok_sack[i].nam:
                self.pok_anzeige.setText('Status von '+ str(trainer.pok_sack[i].nam)+ ':\n'
                                     'Angriff: '+ str(trainer.pok_sack[i].ang)+ '\n'
                                     'Kritisch: '+ str(trainer.pok_sack[i].krit)+ '\n'
                                     'Verteidigung: '+ str(trainer.pok_sack[i].vert)+'\n'
                                     'Leben: '+ str(trainer.pok_sack[i].leb)+ 'von' +  str(trainer.pok_sack[i].lebenges)+'\n'
                                     'Exp: '+ str(trainer.pok_sack[i].exp)+ 'von'+ str(trainer.pok_sack[i].exp_needed)+'\n'
                                     'Level:'+ str(trainer.pok_sack[i].level)+'\n'
                                     'Element:'+str(trainer.pok_sack[i].element))
                
                if trainer.pok_sack[i].exp > trainer.pok_sack[i].exp_needed: 
                    trainer.pok_sack[i].level_up()
                    self.label.setText('Juhuuu, '+str(trainer.pok_sack[i].nam)+ ' hat Level '+ str(trainer.pok_sack[i].level)+ ' erreicht!')
                    self.label.adjustSize()
                if trainer.pok_sack[i].level == 3:
                    self.label.setText(str(trainer.pok_sack[i].nam)+ 'entwickelt sich!')
                    self.label.adjustSize()
                    trainer.pok_sack[i].evolve1()
                self.pok_anzeige.move(700,70)
                self.pok_anzeige.adjustSize()
                self.pok_anzeige.show()
            else:
                pass
    def start_thread(self):
        self.t1 = my_thread(1,'t1')
        self.t1.start()
        
    def stop_thread(self):
        self.t1.stop()
    
    def pok_sack_combo(self):
        self.combo_Box = QComboBox(self)
        for i in range(0,len(trainer.pok_sack)):
            self.combo_Box.addItem(trainer.pok_sack[i].nam)
        self.combo_Box.move(700,30)
        self.combo_Box.show()
    
    def trank(self):
        self.index_pok_heilen = self.combo_Box.currentIndex()
        if trainer.inventar['Trank'][0] >= 1:
            trainer.heile_pok(self.index_pok_heilen)
        else:
            self.label.setText('Keine Tränke in deinem Inventar')
            self.label.adjustSize()
        self.pok_anzeige.adjustSize()
    

In [ ]:
app = QApplication(sys.argv)
w = Fenster()
sys.exit(app.exec_())


    